In [4]:
! pip install -r  /content/drive/MyDrive/playing-with-LLM/llama/requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-icua_wwb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-icua_wwb
  Resolved https://github.com/huggingface/peft.git to commit 9f7492577ff91c51077308f98dade45bf32c268a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-r9qqt3jw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-r9qqt3jw
  Resolved https://github.com/huggingface/transformers.git to commit 66a378429d0e085e4e72bc63a4147889a3b65a14
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━

In [6]:
import os
import sys
import fire
import gradio as gr
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [9]:
load_8bit: bool = False
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained(
        "decapoda-research/llama-7b-hf",
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map="auto",
    )

if not load_8bit:
    model.half()  # seems to fix bugs for some users.

model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [16]:
def evaluate(
    instruction,
    input=None,
    max_new_tokens=128,
    num_beams=1,
    do_sample=False,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    **kwargs,
):
    print('do_sample: %s' % str(do_sample))
    prompt = instruction
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        num_beams=num_beams,
        do_sample=do_sample,
        temperature=temperature,
        # top_k=top_k,
        top_p=top_p,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output




In [17]:
instruction = 'write a story'
response = evaluate(
    instruction,
    max_new_tokens=128,
    num_beams=1,
    do_sample=True,
    temperature=2.0,
    top_p=0.9,
    top_k=40,
    )
print(response)


do_sample: True
 ⁇  write a story! 2 hours (approx.). The writer might then make further revisions based on the discussion during the feedback meeting — or not! — before handing her story back to the teacher for the second (formal) feeddbaaaak.
Dare #1 will help you discover things that aren’t (quite) working and will hopefully bring your story further along so this can happen quickly. This feedback-dance isn’t so easy to accomplish — a good student-critique can quickly turn bad when feedback comes from only the few students who have always had great feedback themselves in their work and don’t


In [19]:
server_name = "0.0.0.0"
share_gradio = True
gr.Interface(
    fn=evaluate,
    inputs=[
        gr.components.Textbox(
            lines=2,
            label='Instruction',
        ),
        gr.components.Textbox(lines=2, label="Input", placeholder="none"),
        gr.components.Slider(
            minimum=1, maximum=2000, step=1, value=128, label="Max tokens"
        ),
        gr.components.Slider(
            minimum=1, maximum=4, step=1, value=4, label="Beams"
        ),
        gr.components.Checkbox(value=False, label='Do sample'),
        gr.components.Slider(
            minimum=0, maximum=25, value=0.1, label="Temperature"
        ),
        gr.components.Slider(
            minimum=0, maximum=1, value=0.75, label="Top p"
        ),
        gr.components.Slider(
            minimum=0, maximum=100, step=1, value=40, label="Top k"
        ),
    ],
    outputs=[
        gr.inputs.Textbox(
            lines=5,
            label="Output",
        )

    ],
    title='LLama',
    description='LLama.'
).launch(server_name=server_name, share=share_gradio,debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9d31b1cb1b480ae60f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


do_sample: False
do_sample: False
do_sample: False
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://e07f686256e4455863.gradio.live
Killing tunnel 0.0.0.0:7861 <> https://d19b0f4db668aa972a.gradio.live
Killing tunnel 0.0.0.0:7862 <> https://54d146c85695088fd9.gradio.live
Killing tunnel 0.0.0.0:7863 <> https://205e7ea03092da174d.gradio.live
Killing tunnel 0.0.0.0:7864 <> https://9d31b1cb1b480ae60f.gradio.live
